In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers, callbacks
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Flatten, Dense, Dropout
%matplotlib inline
import itertools

Using TensorFlow backend.


### Analyzing data to start classification

In [2]:
#Generating Random Forest for data
test_size = 0.2
random=30
df = pd.read_csv('Downloads/oasis_cross-sectional.csv',index_col='ID')
#df.head()

df = df[df['CDR']!=2]
#Unmark the following line to take off in the binary classification
#df = df[df['CDR']!=0.5]

#RETURN TO GROUP 0.5 AND 1
df['CDR'] = df['CDR'].apply(lambda x: 1 if x==0.5 else x)
print(df['CDR'].value_counts())
y = df['CDR'].apply(int)
X = df.drop('Hand',axis=1) #all subjects use right hand


#binaring gender
X = pd.concat([X, pd.get_dummies(X['M/F'])['M']],axis=1).drop('M/F',axis=1)

#converting nWBV and ASF to one-measure scale
def conversor_1000(toBeConverted):
    if toBeConverted < 10:
        return toBeConverted*(10**3)
    else:
        return toBeConverted
    
X['nWBV'] = X['nWBV'].apply(conversor_1000)
X['ASF'] = X['ASF'].apply(conversor_1000)

1.0    98
0.0    98
Name: CDR, dtype: int64


In [3]:
def progressive_sampling(xLabel,yLabel):
    lre = LinearRegression()
    best_combination = []
    r2 = 10000
    valid = []


    for a in xLabel:
        valid.append(a)
    
    for i in range(len(xLabel)-1):
        print('vez=',i,'\n')
        entrou = False
        for j in valid:                              
            best_combination.append(j)
            #print(xr_train[best_combination].head())
            #print(yr_train.head())
            lre.fit(xr_train[best_combination],yr_train)
            pre = lre.predict(xr_test[best_combination])
            print(best_combination,' = ',mean_squared_error(yr_test,pre))
            best_combination.remove(j)
            if mean_squared_error(yr_test,pre) < r2:
                r2 = mean_squared_error(yr_test,pre)
                value = j
                entrou=True
        if entrou:        
            best_combination.append(value)
            print(value)
            valid.remove(value)

    return best_combination

yLabel = 'SES'
Xra = X[~X['SES'].isnull()]    
xr_train, xr_test, yr_train, yr_test = train_test_split(Xra.drop(yLabel,axis=1),Xra[yLabel], test_size=test_size,random_state=random)

teste=np.array(X.drop('SES',axis=1).columns).ravel()
best_predictors = progressive_sampling(teste,'SES')            
#sns.pairplot(X[~X['SES'].isnull()],hue='CDR')

('vez=', 0, '\n')
(['Age'], ' = ', 1.5606177310065883)
(['Educ'], ' = ', 0.5840096484598537)
(['MMSE'], ' = ', 1.438508182110793)
(['CDR'], ' = ', 1.5568804534541223)
(['eTIV'], ' = ', 1.4690132665112507)
(['nWBV'], ' = ', 1.5812759185864671)
(['ASF'], ' = ', 1.4786001437607759)
(['M'], ' = ', 1.5468360233695195)
Educ
('vez=', 1, '\n')
(['Educ', 'Age'], ' = ', 0.58331191874241295)
(['Educ', 'MMSE'], ' = ', 0.58390839573466957)
(['Educ', 'CDR'], ' = ', 0.5929668850055807)
(['Educ', 'eTIV'], ' = ', 0.57170397811704321)
(['Educ', 'nWBV'], ' = ', 0.58827734881585114)
(['Educ', 'ASF'], ' = ', 0.56990011196591928)
(['Educ', 'M'], ' = ', 0.58487032885200751)
ASF
('vez=', 2, '\n')
(['Educ', 'ASF', 'Age'], ' = ', 0.57358078491039466)
(['Educ', 'ASF', 'MMSE'], ' = ', 0.56943543616879866)
(['Educ', 'ASF', 'CDR'], ' = ', 0.58177411014732772)
(['Educ', 'ASF', 'eTIV'], ' = ', 0.56831774146286118)
(['Educ', 'ASF', 'nWBV'], ' = ', 0.57666412523673793)
(['Educ', 'ASF', 'M'], ' = ', 0.57795509019449409)

In [4]:
#setting missing data from SES using linear regression

#RETURN BECAUSE ITS WORKING
lr = LinearRegression()
lr.fit(Xra[best_predictors],Xra[yLabel])
predicted_r = lr.predict(X[X['SES'].isnull()][best_predictors])
updatex = pd.Series(predicted_r,index=X[X['SES'].isnull()].index)
X['SES'].update(updatex)
print(X)



#LEFT THIS WAY

#X[X['SES'].isnull()]['SES'] = update

#print(X)
#def insert_SES():
    
#predicted_r



##Testes
##X[X['SES'].isnull()]
##np.hstack((predicted_r.reshape((len(predicted_r),1)),yr_test.reshape((len(predicted_r),1))))

##print(mean_squared_error(yr_test,predicted_r))



               Age  Educ       SES  MMSE  CDR  eTIV   nWBV     ASF  M
ID                                                                   
OAS1_0072_MR1   60     5  1.000000    30  0.0  1402  823.0  1252.0  0
OAS1_0200_MR1   60     2  4.000000    30  0.0  1366  807.0  1285.0  0
OAS1_0109_MR1   61     4  3.000000    30  0.0  1313  813.0  1337.0  0
OAS1_0455_MR1   61     2  4.000000    28  0.0  1354  825.0  1297.0  0
OAS1_0456_MR1   61     5  2.000000    30  0.0  1637  780.0  1072.0  1
OAS1_0114_MR1   62     2  4.000000    30  0.0  1378  804.0  1274.0  1
OAS1_0457_MR1   62     3  3.000000    26  0.0  1372  766.0  1279.0  0
OAS1_0070_MR1   63     3  2.000000    30  0.0  1327  801.0  1323.0  0
OAS1_0078_MR1   64     3  2.000000    30  0.0  1395  809.0  1258.0  0
OAS1_0135_MR1   64     2  4.000000    29  0.0  1561  801.0  1124.0  1
OAS1_0292_MR1   64     4  2.000000    30  0.0  1415  766.0  1240.0  0
OAS1_0030_MR1   65     2  3.000000    29  0.0  1392  764.0  1261.0  0
OAS1_0133_MR1   65  

In [9]:
#using z-score
ss = StandardScaler()
scaled = ss.fit_transform(X.drop(['CDR'],axis=1))
X_zscore = pd.DataFrame(scaled,columns=X.drop(['CDR'],axis=1).columns)
#print(X_zscore.head())
X_train, X_test, y_train, y_test = train_test_split(X_zscore, y.apply(str), test_size=test_size, random_state=random)

In [12]:
def cross_val_subset(x,y,k=10):
    from random import Random
    from itertools import cycle
    temp_s = list(zip(x,y))
    Random(42).shuffle(temp_s)
    x,y = zip(*temp_s)
    
    TEST = int(len(y)*0.1)
    TRAIN = len(y)-TEST
    
    rtn_x,rtn_y = np.zeros(k,dtype=tuple), np.zeros(k,dtype=tuple)
    
    for i in range(k):
        start = i*TEST
        np.roll(x,start); np.roll(y,start)
        rtn_x[i] = (x[0:TRAIN],x[TRAIN:])
        rtn_y[i] = (y[0:TRAIN],y[TRAIN:])
    return rtn_x, rtn_y

In [13]:
def super_learner(x,y,x_test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC

    rf = RandomForestClassifier(bootstrap=True, class_weight=None,n_estimators=500, criterion='gini')
    nb = GaussianNB()
    svm = SVC()
    
    k_x,k_y = cross_val_subset(x,y)
    
    level_one_l = []
    SL_rf, SL_nb, SL_svm = [],[],[]
    classifiers = [rf,nb,svm]
    features = [SL_rf, SL_nb,SL_svm]
    y_labels = [] 
    
    for fold_x, fold_y in zip(k_x,k_y):
        [i.fit(fold_x[0],fold_y[0]) for i in classifiers]
        [f.append(c.predict(fold_x[1])) for c,f in zip(classifiers,features)]
        y_labels.append(fold_y[1])
    [level_one_l.append(np.array(f).ravel()) for f in features]
    f_len = len(level_one_l[0])
    level_one = np.hstack([np.reshape(l,(f_len,1)) for l in level_one_l])
    level_one = np.hstack([level_one,np.reshape(np.array(y_labels),(f_len,1))]).astype(float)
    
    test_f = [c.predict(x_test) for c in classifiers]
    t_len = len(test_f[0])
    level_two = np.hstack([np.reshape(t,(t_len,1)) for t in test_f]).astype(float)
    
    l2 = SVC()
    l2.fit(level_one[:,:3],level_one[:,3])
    
    prediction = l2.predict(level_two)
    
    return prediction

sl_f = super_learner(X_train.values, y_train.values,X_test.values)
sl_f

array([ 1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,
        0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,
        1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.])

In [15]:
print(classification_report(sl_f,y_test.values.astype(float)))

             precision    recall  f1-score   support

        0.0       0.82      0.82      0.82        17
        1.0       0.87      0.87      0.87        23

avg / total       0.85      0.85      0.85        40



In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

rf = RandomForestClassifier(bootstrap=True, class_weight=None,n_estimators=500, criterion='gini')
nb = GaussianNB()
svm = SVC()

rf.fit(X_train.values,y_train.values)
nb.fit(X_train.values,y_train.values)
svm.fit(X_train.values,y_train.values)

pr = rf.predict(X_test)
pn = nb.predict(X_test)
ps = svm.predict(X_test)

print(classification_report(pr,y_test.values))
print(classification_report(pn,y_test.values))
print(classification_report(ps,y_test.values))

             precision    recall  f1-score   support

          0       0.88      0.75      0.81        20
          1       0.78      0.90      0.84        20

avg / total       0.83      0.82      0.82        40

             precision    recall  f1-score   support

          0       0.94      0.76      0.84        21
          1       0.78      0.95      0.86        19

avg / total       0.87      0.85      0.85        40

             precision    recall  f1-score   support

          0       0.94      0.73      0.82        22
          1       0.74      0.94      0.83        18

avg / total       0.85      0.82      0.82        40



In [23]:
df = pd.DataFrame(np.vstack([np.reshape(pr,(len(pr),1)),np.reshape(pn,(len(pr),1)),np.reshape(ps,(len(pr),1))]))
print(df.corr)

<bound method DataFrame.corr of      0
0    1
1    1
2    0
3    1
4    1
5    0
6    1
7    1
8    0
9    0
10   0
11   1
12   0
13   0
14   1
15   0
16   1
17   0
18   1
19   1
20   0
21   0
22   1
23   0
24   0
25   1
26   1
27   0
28   0
29   1
..  ..
90   0
91   1
92   0
93   0
94   1
95   0
96   1
97   0
98   1
99   1
100  0
101  0
102  1
103  1
104  0
105  0
106  1
107  0
108  0
109  1
110  1
111  0
112  0
113  1
114  0
115  0
116  0
117  0
118  0
119  1

[120 rows x 1 columns]>
